In [1]:
import numpy as np
import netCDF4 as nc
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.io as scio
import cmaps
from mpl_toolkits.basemap import Basemap, cm
from scipy import stats
from scipy.signal import detrend
from datetime import datetime

# 逐点detrend后SST.EOF前4个和T300.EOF 前3个 (20˚NS)，减去EOF4（9个member）SST.EOF (全场，但只加热带部分）之后（eof'）的第一mode（eof'1* pc'1[0]与每一个PC4相乘之后除以权重）

## 读取数据

In [2]:
ipth_in = '/Users/chenyipeng/Desktop/cyp_work/cwork2020/nature/enso_0/input/'
ipth_in_eof = '/Users/chenyipeng/Desktop/cyp_work/cwork2020/nature/analysis/eof2_detrend_eof/'
ipth = '/Users/chenyipeng/Desktop/cyp_work/cwork2021/nature/eof_deof4_allarea_4/'
ipth_eof_same = '/Users/chenyipeng/Desktop/cyp_work/cwork2020/nature/analysis/deof4_cmip_9_12_eof_all_area/'     

In [3]:
cmip = Dataset(ipth_in+'CMIP5.input.36mon.1861_2001.nc')
lon_cmip = np.array(cmip.variables['lon'])
lat_cmip = np.array(cmip.variables['lat'])
lev_cmip = np.array(cmip.variables['lev'])
time_ = np.linspace(1861,2004,1716)

## SST.EOF1-4和T300.EOF1-3减去9member共性（SST.eof'1）

In [4]:
lat_cmip1 = np.where(lat_cmip>=-20)[0][0]
lat_cmip2 = np.where(lat_cmip<=20)[0][-1]

In [5]:
# 9member SST的EOF模态
sst_eof = [0,1,2,3]
eof = np.zeros([9,72,1])
eofs_9same = np.zeros([72,24,1])
pc = np.zeros([1,1716])
enso_sst9_eof4_weight = np.zeros([9,9,72,1716])
enso_sst9_same_weight = np.zeros([9,9,72,1716])

data = scio.loadmat(ipth_in_eof+'eof16_detrend_cmip5_9_12member.mat')
pcs_sst = data['PCs_sst']
eofs = data['EOFs_sst']
eofs_sst = eofs.swapaxes(1, 0)
for i in range(9):
    for j in sst_eof:
        eof[:,:,0] = eofs_sst[:,:,j,i]
        pc[0,:] = pcs_sst[:,j,i]
        enso_sst9_eof4_weight[i,:,:,:] += np.dot(eof,pc)
        
data_ = scio.loadmat(ipth_eof_same+'deof4_cmip5_eof_all_area.mat')
eofs_9same[:,:,0] = data_['EOFs_sst9'][:,:,0]
pc_sst_9same = data_['PCs_sst9'][:,0] #EOF'的第一模态的pc'1
eofs_sst_9same = eofs_9same.swapaxes(1, 0)
for i in range(9):
    pc[0,:] = pcs_sst[:,3,i]
    enso_sst9_same_weight[i,:,:,:] = np.dot(
        eofs_sst_9same*pc_sst_9same[i],pc)[lat_cmip1:lat_cmip2+1,:,:]
    
enso_sst9_weight = enso_sst9_eof4_weight - enso_sst9_same_weight
#除以权重        
arw_ = np.tile(np.cos(np.radians(lat_cmip[lat_cmip1:lat_cmip2+1])),
               (enso_sst9_weight.shape[0],
                enso_sst9_weight.shape[2],enso_sst9_weight.shape[3],1))
arw = arw_.swapaxes(2,3).swapaxes(1,2)
enso_sst9 = enso_sst9_weight/arw

#enso_sst9[9,9,72,1716]变为enso_sst9_new[1269,36,9,72]
enso_sst9_new = np.zeros([1269,36,9,72])
i = 0
for n in range(enso_sst9.shape[0]): #9
    for j in range(141):
        nyear = j*12
        enso_sst9_new[i,:,:,:] = enso_sst9.swapaxes(3, 1).swapaxes(3, 2)[n,nyear:nyear+36,:,:]
        i = i+1

In [6]:
# 9member t300 的EOF模态 
t300_eof = [0,1,2]
eof = np.zeros([9,72,1])
pc = np.zeros([1,1716])
enso_ohc9_weight = np.zeros([9,9,72,1716])
data = scio.loadmat(ipth_in_eof+'eof16_detrend_cmip5_9_12member.mat')
pcs_ohc = data['PCs_ohc']
eofs = data['EOFs_ohc']
eofs_ohc = eofs.swapaxes(1, 0)
for i in range(9):
    for j in t300_eof:
        eof[:,:,0] = eofs_ohc[:,:,j,i]
        pc[0,:] = pcs_ohc[:,j,i]
        enso_ohc9_weight[i,:,:,:] += np.dot(eof,pc)
        
#除以权重
enso_ohc9 = enso_ohc9_weight/arw

#enso_ohc9[9,9,72,1716]变为enso_ohc9_new[1269,36,9,72]
enso_ohc9_new = np.zeros([1269,36,9,72])
i = 0
for n in range(enso_ohc9.shape[0]): #9
    for j in range(141):
        nyear = j*12
        enso_ohc9_new[i,:,:,:] = enso_ohc9.swapaxes(3, 1).swapaxes(3, 2)[n,nyear:nyear+36,:,:]
        i = i+1

In [7]:
data_0 = np.zeros([1269,36,1,72])
sst_enso_cmip_new0 = np.concatenate((data_0,enso_sst9_new),axis=2)
sst_enso_cmip_new1 = np.concatenate((sst_enso_cmip_new0,data_0),axis=2)
sst_enso_cmip_new = np.concatenate((sst_enso_cmip_new1,data_0),axis=2)

ohc_enso_cmip_new0 = np.concatenate((data_0,enso_ohc9_new),axis=2)
ohc_enso_cmip_new1 = np.concatenate((ohc_enso_cmip_new0,data_0),axis=2)
ohc_enso_cmip_new = np.concatenate((ohc_enso_cmip_new1,data_0),axis=2)

In [8]:
##create NetCDF file
cmip_new = nc.Dataset(ipth+'input/CMIP5.input.36mon.1861_2001.nc', 'w', format='NETCDF4')

##define dimesions
long = cmip_new.createDimension('lon', size=72)
lati = cmip_new.createDimension('lat', size=12)
levl = cmip_new.createDimension('lev', size=36)
times = cmip_new.createDimension('time', size=1269)

##define variables for storing data
lon = cmip_new.createVariable('lon', 'f8', dimensions='lon')
lat = cmip_new.createVariable('lat', 'f8', dimensions='lat')
lev = cmip_new.createVariable('lev', 'f8', dimensions='lev')
sst1 = cmip_new.createVariable('sst1', 'f4', dimensions=('time','lev','lat','lon'))
t300 = cmip_new.createVariable('t300', 'f4', dimensions=('time','lev','lat','lon'))

##add data to variables
lon[:] = lon_cmip
lat[:] = lat_cmip[lat_cmip1-1:lat_cmip2+3]
lev[:] = lev_cmip
sst1[:,:,:,:] = sst_enso_cmip_new
t300[:,:,:,:] = ohc_enso_cmip_new

##add attributes
#add global attributes
cmip_new.title = 'CMIP5.input.36mon.1861_2001.nc__tropical_SST20NS_20N20S'
cmip_new.history = 'Created: ' + datetime(2021, 12, 28, 17, 48, 0).strftime('%Y-%m-%d %H:%M:%S')

cmip_new.close()